In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '')
ds_price['STOCK_CLOSE'] = pd.to_numeric(ds_price['STOCK_CLOSE'])
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_DATE'].dt.to_period("Y")
ds_price = ds_price.merge(ds_ratio, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='left')
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).first()

ds_price['RETURN'] = ds_price['STOCK_CLOSE'].div(ds_price.groupby(level=0)['STOCK_CLOSE'].shift(1))
ds_price['EP'] = ds_price['STOCK_DATA'].div(ds_price['STOCK_CLOSE'])
#ds_price['QUANTILES'] = ds_price.groupby('STOCK_YEAR')['EP'].transform(pd.qcut,10,list(range(10), duplicates='drop'))
ds_price

TypeError: list() takes at most 1 argument (2 given)